# Data Understanding and Cleaning

The aim of the following notebook is to understand the structure and identify potencial data quality issues that could affect for further exploration data analysis and RFM calculations.

The notebook is divided in the following sub-sections:
- Dataset dimensions
- Dataset colum data-type and meaning
- Missing values
- Value ranges for numeric columns
- Duplicate values
- Column type conversions for RFM analysis preparation

First it's necessary to import the Python libraries needed for this initial data exploration. In the same cell, the path of the file and the method used to read it are included.

In [2]:
import pandas as pd

path = "../data_raw/online_retail_II.csv"
df = pd.read_csv(path)

## Dataset dimensions
According to the result shown below, there is a rectangular dataset with 1,067,371 instances of height and 8 columns of width.

In [3]:
df.shape

(1067371, 8)

## Dataset colum data-type and meaning
The dataset is compound of the following 8 columns:
| Column | Data Type | Description |
|---|---|---|
| Invoice | Nominal, String | A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. |
| StockCode | Nominal, String | A 5-digit integral number uniquely assigned to each distinct product. |
| Description | Nominal, String | Product item name. |
| Quantity | Numeric, Int64 | The quantities of each product (item) per transaction. |
| InvoiceDate | Numeric, String | The day and time when a transaction was generated. |
| Price | Numeric, Float64 | Product price per unit in sterling |
| Customer ID | Nominal, Float64 | A 5-digit integral number uniquely assigned to each customer. |
| Country | NOminal, String | The name of the country where a customer resides. |

## Missing values
However, not all the attributes above have their entire registries with not null values, as we can see in the cell result below. Both _Description_ and _Customer ID_ have some rows with null values, being the last one the column with the highest count of null values.

Due to the main goal of the project, which is to implement a RFM analysis, the _Description_ column will be discharged because its lack of relevance for this specific type of analysis. Moreover, _Customer ID_'s null values will be removed in the cleaning process, including the rest of the attributes of the corresponding rows, since RFM analysis requires customer level identification. This decision reduces the total number of records but ensures accurate and meaningful customer segmentation.

In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  str    
 1   StockCode    1067371 non-null  str    
 2   Description  1062989 non-null  str    
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  str    
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  str    
dtypes: float64(2), int64(1), str(5)
memory usage: 65.1 MB


### Cleaning Customer ID missing values
Before continue with the rest of the data understanding process, transactions without a valid _Customer ID_ value will be removed. This is to avoid hindering the analysis of the remaining data.

In [5]:
df_v1 = df.dropna(subset=['Customer ID'])

In [6]:
df_v1.info()

<class 'pandas.DataFrame'>
Index: 824364 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      824364 non-null  str    
 1   StockCode    824364 non-null  str    
 2   Description  824364 non-null  str    
 3   Quantity     824364 non-null  int64  
 4   InvoiceDate  824364 non-null  str    
 5   Price        824364 non-null  float64
 6   Customer ID  824364 non-null  float64
 7   Country      824364 non-null  str    
dtypes: float64(2), int64(1), str(5)
memory usage: 56.6 MB


## Value ranges for numeric columns
For the numerical values, it is important to know what the extreme values of the ranges are and if they are valid according to the business logic. Therefore, after calling the describe function, we found the minimum value for _Quantity_ was negative, which could represent devolutions or cancellations; additionally, the lowest value of the _Price_ column is zero, which doesn't represent a real monetary transaction.

Overall, _Quantity_ negative values and _Price_ zero values must be excluded from the analysis to prevent distorsions in the recency, frequency and monetary values of the customers.

In [7]:
df_v1.describe()

,Quantity,Price,Customer ID
count,824364.000000,824364.000000,824364.000000
mean,12.414574,3.676800,15324.638504
std,188.976099,70.241388,1697.464450
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13975.000000
50%,5.000000,1.950000,15255.000000
75%,12.000000,3.750000,16797.000000
max,80995.000000,38970.000000,18287.000000


In [8]:
quantity_sorted_df = df_v1.sort_values(by="Quantity")
quantity_sorted_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1065883,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom
587085,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom
529729,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277.0,France


In [9]:
price_sorted_df = df_v1.sort_values(by="Price")
price_sorted_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
248583,513416,22423,REGENCY CAKESTAND 3 TIER,5,2010-06-24 12:34:00,0.0,13089.0,United Kingdom
392008,527084,22630,DOLLY GIRL LUNCH BOX,64,2010-10-14 15:33:00,0.0,14646.0,Netherlands
16107,490727,M,Manual,1,2009-12-07 16:38:00,0.0,17231.0,United Kingdom
612250,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
945865,572893,21208,PASTEL COLOUR HONEYCOMB FAN,5,2011-10-26 14:36:00,0.0,18059.0,United Kingdom


### Cleaning Quantity and Price minimum values
Before continue with the rest of the data understanding process, transactions with a negative quantity or a zero price will be removed. This is to avoid hindering the analysis of the remaining data.

In [10]:
df_v2 = df_v1[(df_v1["Quantity"] > 0) & (df_v1["Price"] > 0)]

In [11]:
df_v2.shape

(805549, 8)

In [12]:
df_v2.min()

Invoice                                     489434
StockCode                                    10002
Description      DOORMAT UNION JACK GUNS AND ROSES
Quantity                                         1
InvoiceDate                    2009-12-01 07:45:00
Price                                        0.001
Customer ID                                12346.0
Country                                  Australia
dtype: object

In [13]:
df_v2.max()

Invoice                                  581587
StockCode                               TEST002
Description    ZINC WIRE SWEETHEART LETTER TRAY
Quantity                                  80995
InvoiceDate                 2011-12-09 12:50:00
Price                                   10953.5
Customer ID                             18287.0
Country                             West Indies
dtype: object

## Duplicate values
In transactional datasets, duplicate values are managed differently from a regular dataset. Transactions with different _Invoice_ values but identical product, quantity and price, represent a different purchase and should not be removed from the dataset. However, rows with their entire column values the same, including _Invoice_ values, are considered duplicates and must be deleted.

In [14]:
col = ['Invoice', 'StockCode', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']
df_v2.duplicated(subset=col).sum()

np.int64(26126)

In [15]:
df_v2[df_v2.duplicated(subset=col, keep=False)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
362,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
363,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
365,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
367,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,16329.0,United Kingdom
368,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067136,581538,22068,BLACK PIRATE TREASURE CHEST,1,2011-12-09 11:34:00,0.39,14446.0,United Kingdom
1067150,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
1067153,581538,22992,REVOLVER WOODEN RULER,1,2011-12-09 11:34:00,1.95,14446.0,United Kingdom
1067160,581538,22694,WICKER STAR,1,2011-12-09 11:34:00,2.10,14446.0,United Kingdom


Sample of duplicate rows:

In [16]:
df_v2[(df_v2["Invoice"] == "489517") & (df_v2["StockCode"] == "21912")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
363,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
394,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom


### Cleaning Duplicate Rows
Before to continue with the rest of the data understanding process, duplicate transactions will be removed. This is to avoid hindering the analysis of the rest of the data.

In [19]:
df_v3 = df_v2.drop_duplicates(subset=col)

In [20]:
df_v3.shape

(779423, 8)

In [23]:
# Verification of the cleaning process
df_v3.duplicated(subset=col).sum()

np.int64(0)

## Column type conversions for RFM analysis preparation
Having dataset format suitable for customer level analysis is key. Based on the **Dataset column data-type and meaning** section of this notebook, the _InvoiceDate_ column is originally stored as a string type, which need to be parsed to DateTime type for easier Recency calculations such as sorting transactions and aggregating customer activity over time. 

Additionally, the _CustomerID_ values are Float64 type, so they will be converted to Int64 type to decrease storage space and ensure consistency.

Finally, _Total Price_ column (with Float64 data-type) will be included for easier Monetary calculations. This column values are created by multiplying _Quantity_ and _Price_.

In [24]:
df_v4 = df_v3.copy()

In [28]:
df_v4['InvoiceDate'] = pd.to_datetime(df_v4['InvoiceDate'])

In [32]:
df_v4['Customer ID'] = df_v4['Customer ID'].astype('Int64')

In [33]:
df_v4['TotalPrice'] = df_v4['Quantity'] * df_v4['Price']

Verification of the conversion process:

In [34]:
df_v4.info()

<class 'pandas.DataFrame'>
Index: 779423 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      779423 non-null  str           
 1   StockCode    779423 non-null  str           
 2   Description  779423 non-null  str           
 3   Quantity     779423 non-null  int64         
 4   InvoiceDate  779423 non-null  datetime64[us]
 5   Price        779423 non-null  float64       
 6   Customer ID  779423 non-null  Int64         
 7   Country      779423 non-null  str           
 8   TotalPrice   779423 non-null  float64       
dtypes: Int64(1), datetime64[us](1), float64(2), int64(1), str(4)
memory usage: 60.2 MB


In [36]:
df_v4.head(3)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
